## COLLABORATIVE FILTERING AS A MODEL OF GROUP DECISION-MAKING

A collaborative filtering algorithm usually works by searching a large group of people and finding a smaller set with tastes similar to yours. It looks at other things they like and combines them to create a ranked list of suggestions. There are several different ways of deciding which people are similar and combining their choices to make a list ; this tutorial cover a few of these.



In [208]:
#import libraries
from pulp import *
import pandas as pd
import numpy as np
from numpy.linalg import norm
import math
import random
from scipy import stats
from scipy import spatial

### Which movie we can suggest to Anne?

Our objective is to provide recommendations to Anne, taking into account the ratings of the seven movie critics. All the data are stored in an Excel file.


In [209]:
#reading file
df = pd.read_csv(r'data/movie.csv', delimiter=',')
df

,Person,Lady,Snakes,Luck,Superman,Dupree,Night
0,Lisa Rose,2.5,3.5,3.0,3.5,2.5,3.0
1,Gene Seymour,3.0,3.5,1.5,5.0,3.5,3.0
2,Michael Phillips,2.5,3.0,NaN,3.5,NaN,4.0
3,Claudia Puig,NaN,3.5,3.0,4.0,2.5,4.5
4,Mick Lasalle,3.0,4.0,2.0,3.0,2.0,3.0
5,Jack Matthews,3.0,4.0,NaN,5.0,3.5,3.0
6,Toby,NaN,4.5,NaN,4.0,1.0,NaN
7,Anne,1.5,NaN,4.0,NaN,2.0,NaN


#### 1. Build a dictionary Critiques 
From the Excel file, build a dictionary critiques containing the movie critics and their ratings (of course, you
can choose another data structure). 

In [210]:
#change rows to column
df= df.transpose()

In [211]:
df = df.rename(columns=df.iloc[0])

In [212]:
df = df.drop(df.index[0])

In [213]:
df

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick Lasalle,Jack Matthews,Toby,Anne
Lady,2.5,3.0,2.5,NaN,3.0,3.0,NaN,1.5
Snakes,3.5,3.5,3.0,3.5,4.0,4.0,4.5,NaN
Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN,4.0
Superman,3.5,5.0,3.5,4.0,3.0,5.0,4.0,NaN
Dupree,2.5,3.5,NaN,2.5,2.0,3.5,1.0,2.0
Night,3.0,3.0,4.0,4.5,3.0,3.0,NaN,NaN


In [214]:
Critiques = df.to_dict()

In [215]:
print(Critiques)

{'Lisa Rose': {'Lady': 2.5, 'Snakes': 3.5, 'Luck': 3.0, 'Superman': 3.5, 'Dupree': 2.5, 'Night': 3.0}, 'Gene Seymour': {'Lady': 3.0, 'Snakes': 3.5, 'Luck': 1.5, 'Superman': 5.0, 'Dupree': 3.5, 'Night': 3.0}, 'Michael Phillips': {'Lady': 2.5, 'Snakes': 3.0, 'Luck': nan, 'Superman': 3.5, 'Dupree': nan, 'Night': 4.0}, 'Claudia Puig': {'Lady': nan, 'Snakes': 3.5, 'Luck': 3.0, 'Superman': 4.0, 'Dupree': 2.5, 'Night': 4.5}, 'Mick Lasalle': {'Lady': 3.0, 'Snakes': 4.0, 'Luck': 2.0, 'Superman': 3.0, 'Dupree': 2.0, 'Night': 3.0}, 'Jack Matthews': {'Lady': 3.0, 'Snakes': 4.0, 'Luck': nan, 'Superman': 5.0, 'Dupree': 3.5, 'Night': 3.0}, 'Toby': {'Lady': nan, 'Snakes': 4.5, 'Luck': nan, 'Superman': 4.0, 'Dupree': 1.0, 'Night': nan}, 'Anne': {'Lady': 1.5, 'Snakes': nan, 'Luck': 4.0, 'Superman': nan, 'Dupree': 2.0, 'Night': nan}}


In [216]:
print(Critiques['Lisa Rose'])

{'Lady': 2.5, 'Snakes': 3.5, 'Luck': 3.0, 'Superman': 3.5, 'Dupree': 2.5, 'Night': 3.0}


### 2. Calculating Similarity score
After collecting data about the things people like, you need a way to determine how similar people are in their tastes.
You do this by comparing each person with every other person and calculating a similarity score. In our case, we
should determine which persons are “similar” to Anne.

#### 2(a) To compute a simple similarity score, we use the Manhattan distance or an Euclidean distance.

##### Manhattan Distance

In [217]:
def sim_distanceManhattan(person1,person2):
    distance=0
    for movie, rating1 in person1.items():
        if (pd.isnull(rating1) or pd.isnull(person2[movie])): 
            distance+= 0
        else:
            rating2=person2[movie]
            difference = rating2 - rating1
            absolute_difference = abs(difference)
            distance += absolute_difference        
    return distance

In [218]:
sim_distanceManhattan(Critiques['Lisa Rose'],
Critiques['Gene Seymour'])

4.5

##### Euclidean Distance

In [219]:
def sim_distanceEuclidienne(person1,person2):
    distance=0
    for movie, rating1 in person1.items():
        if (pd.isnull(rating1) or pd.isnull(person2[movie])): 
            distance+= 0
        else:
            rating2=person2[movie]
            sum2= np.sum(np.square(rating1 - rating2))
            distance += sum2        
    return np.sqrt(distance)

In [220]:
sim_distanceEuclidienne(Critiques['Lisa Rose'],
Critiques['Gene Seymour'])

2.3979157616563596

#### 2(b) RecommendNearestNeighbour
Anne could just look at the person who has tastes most similar to her and look for a movie he likes that Anne has
not seen yet. For each distance above, build the function recommendNearestNeighbor(nouveauCritique, Critiques) returning a list of movies to recommend to the user nouveauCritique, based on the tastes of the critic who is close (similar) to him.

In [221]:
def computeNearestNeighbor(nouveauCritique, Critiques):
    distances=[]
    for critique in Critiques:
        if critique!=nouveauCritique:
            distance=sim_distanceManhattan(Critiques[critique], Critiques[nouveauCritique])
            distances.append((distance,critique))
    distances.sort()
    return distances

In [222]:
computeNearestNeighbor('Toby', Critiques)

[(1.0, 'Anne'),
 (2.0, 'Michael Phillips'),
 (2.5, 'Claudia Puig'),
 (2.5, 'Mick Lasalle'),
 (3.0, 'Lisa Rose'),
 (4.0, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [223]:
def recommendNearestNeighbor(nouveauCritique, Critiques):
    recommended_movies=[]
    nearest_neighbor = computeNearestNeighbor(nouveauCritique, Critiques)[0][1]
    print("The nearest neighbour of {} is {} ".format(nouveauCritique,nearest_neighbor))
    nn_movies = dict([i for i in Critiques[nearest_neighbor].items() if not pd.isna(i[1])])
    nc_movies = dict([i for i in Critiques[nouveauCritique].items() if pd.isna(i[1])])
    
    for movie in nn_movies:
        if movie in nc_movies:
            recommended_movies.append((movie,nn_movies[movie]))
    print("The list of movies recommended is",recommended_movies)
    #return recommended_movies

In [224]:
computeNearestNeighbor('Lisa Rose', Critiques)

[(1.5, 'Michael Phillips'),
 (2.0, 'Claudia Puig'),
 (2.5, 'Anne'),
 (3.0, 'Mick Lasalle'),
 (3.0, 'Toby'),
 (3.5, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [225]:
recommendNearestNeighbor('Lisa Rose',Critiques)

The nearest neighbour of Lisa Rose is Michael Phillips 
The list of movies recommended is []


In [226]:
computeNearestNeighbor('Toby', Critiques)

[(1.0, 'Anne'),
 (2.0, 'Michael Phillips'),
 (2.5, 'Claudia Puig'),
 (2.5, 'Mick Lasalle'),
 (3.0, 'Lisa Rose'),
 (4.0, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [227]:
recommendNearestNeighbor('Anne',Critiques)

The nearest neighbour of Anne is Michael Phillips 
The list of movies recommended is [('Snakes', 3.0), ('Superman', 3.5), ('Night', 4.0)]


#### 2(c) Solve the limitations of 2(b)
To solve the previous limitations, you need to score the items by producing a weighted score (global score) that
ranks the critics. The procedure which determine the recommendations to suggest to Anne is describe in two
steps below. For a given movie a, let us denote by C(a) the list of all the critics which gave a rating to a, and x(a)
the rating given to a by the critic x.


##### Global Score by using Euclidienne Distance 

In [228]:
#calculate the global score of each unwatched movies with Euclidienne Distance
def movie_scoreED(person, movie, Critiques):
    global_score = 0
    score = 0
    for i in Critiques.items():
        if not pd.isnull(i[1][movie]):
            global_score += i[1][movie] / (1+sim_distanceEuclidienne(Critiques[person], i[1]))
            score += 1 / (1+sim_distanceEuclidienne(Critiques[person], i[1]))
    return global_score / score

In [229]:
movie_scoreED("Anne", "Night", Critiques)

3.5462185630243064

##### Global Score by using Manhattan Distance

In [230]:
#calculate the global score of each unwatched movies with ManhattanDistance
def movie_score(person, movie, Critiques):
    global_score = 0
    score = 0
    for i in Critiques.items():
        if not pd.isnull(i[1][movie]):
            global_score += i[1][movie] / (1+sim_distanceManhattan(Critiques[person], i[1]))
            score += 1 / (1+sim_distanceManhattan(Critiques[person], i[1]))
    return global_score / score


In [231]:
movie_score("Anne", "Night", Critiques)

3.6071368399770862

##### 2(c) (i) A Bestrecommend by Using Manhattan and Euclidean Distance
From these explanations, by using Manhattan and Euclidean distances, build the function Bestrecommend
suggesting to Anne a recommendation between the three movies “Snakes”, “Superman” and “Night”.

In [232]:
#a fucntion to find a list of unwatched movies
def get_unwatched_movies(person, Critiques):
    unwatched_movies=[]
    for i in Critiques[person].items():
        if pd.isnull(i[1]):
            unwatched_movies.append(i[0])
    return unwatched_movies

In [233]:
get_unwatched_movies('Anne',Critiques)

['Snakes', 'Superman', 'Night']

###### BestRecommend by Euclidean Distance 

In [234]:
def Bestrecommend2(person, Critiques):
    recommend_movies=[]
    for movie in get_unwatched_movies(person, Critiques):
        global_score = movie_scoreED(person, movie, Critiques)
        recommend_movies.append((global_score,movie))
    print(recommend_movies) 
    return sorted(recommend_movies)[-1][1]

In [235]:
Bestrecommend2("Anne", Critiques)

[(3.7039297694065687, 'Snakes'), (3.933028718913866, 'Superman'), (3.5462185630243064, 'Night')]


'Superman'

###### BestRecommend by Manhattan Distance 

In [236]:
def Bestrecommend(person, Critiques):
    recommend_movies=[]
    for movie in get_unwatched_movies(person, Critiques):
        global_score = movie_score(person, movie, Critiques)
        recommend_movies.append((global_score,movie))
    print(recommend_movies)
    return sorted(recommend_movies)[-1][1]

In [237]:
Bestrecommend("Anne", Critiques)

[(3.7102754377162173, 'Snakes'), (3.908627564898196, 'Superman'), (3.6071368399770862, 'Night')]


'Superman'

#### 2(c) (ii) BestrecommendwithExp
By replacing the weights 1/(1+d(x,Anne) in the previous formula, by exp−d(x,Anne), build the function
BestrecommendwithExp suggesting to Anne a recommendation between the three movies “Snakes”,“Superman” and “Night”.

In [238]:
def movie_score_exp(person, movie, Critiques):
    global_score = 0
    score = 0
    for i in Critiques.items():
        if not pd.isnull(i[1][movie]):
            global_score += i[1][movie] * math.exp(-1* sim_distanceManhattan(Critiques[person], i[1]))
            score += math.exp(-1* sim_distanceManhattan(Critiques[person], i[1]))
    return global_score / score

In [239]:
movie_score_exp('Anne','Superman',Critiques)

3.821068742837592

In [240]:
def BestrecommendwithExp(person, Critiques):
    recommend_movies=[]
    for movie in get_unwatched_movies(person, Critiques):
        global_score = movie_score_exp(person, movie, Critiques)
        recommend_movies.append((global_score,movie))
    print(recommend_movies)
    return sorted(recommend_movies)[-1][1]

In [241]:
BestrecommendwithExp("Anne", Critiques)

[(3.699042610620127, 'Snakes'), (3.821068742837592, 'Superman'), (3.927900678942954, 'Night')]


'Night'

#### 2(d) Similarity between people’s interests is to use a Pearson Correlation Coefficient
A slightly more sophisticated way to determine the similarity between people’s interests is to use a Pearson
correlation coefficient. The correlation coefficient is a measure of how well two sets of data fit on a straight line.
The formula for this is more complicated than the Euclidean distance score, but it tends to give better results in
situations where the data is not well normalized for example, if critics’ movie rankings are routinely more harsh
than average. 

In [242]:
#pearson correlation coefficient
def pearson(person1, person2):
    sum_xy = 0
    sum_x, sum_y = 0, 0
    sum_x2, sum_y2 = 0, 0
    n = 0
    for key in person1:
        if key in person2:
            if not (pd.isnull(person1[key]) or pd.isnull(person2[key])):
                n += 1
                x=person1[key]
                y=person2[key]
                sum_xy +=x*y
                sum_x += x
                sum_y += y
                sum_x2 += x**2
                sum_y2 += y**2
    denominator = np.sqrt(sum_x2 - (sum_x**2) / n) * np.sqrt(sum_y2 - (sum_y**2) / n)
    if denominator == 0:
        return 0
    else:
        return (sum_xy - (sum_x * sum_y) / n) / denominator

In [243]:
#testing pearson coefficient
pearson(Critiques['Lisa Rose'],Critiques['Gene Seymour'])

0.39605901719066977

In [244]:
#calculating global score with pearson coeficient
def pearson_movie_score(person, movie, Critiques):
    global_score = 0
    score = 0
    for i in Critiques.items():
        if not pd.isnull(i[1][movie]):
            global_score += i[1][movie] / (1+pearson(Critiques[person], i[1]))
            score += 1 / (1+pearson(Critiques[person], i[1]))
    return global_score / score

In [245]:
def PearsonRecommend(person,Critiques):
    recommend_movies=[]
    for movie in get_unwatched_movies(person, Critiques):
        global_score = pearson_movie_score(person, movie, Critiques)
        recommend_movies.append((global_score,movie))
    print(recommend_movies)
    return sorted(recommend_movies)[-1][1]

In [246]:
PearsonRecommend("Anne", Critiques)

[(3.627402308844829, 'Snakes'), (4.446519851865466, 'Superman'), (3.107685434763305, 'Night')]


'Superman'

#### 2(e) Compute the function CosineRecommend
Suggesting to Anne, by using the Cosine correlation coefficient, a recommendation between the three movies “Snakes”, “Superman” and “Night”. 

In [247]:
def cosineCoefficient(person1,person2):
    for key in person1:
        if key in person2:
            if not (pd.isnull(person1[key]) or pd.isnull(person2[key])):
                x ,y =person1[key], person2[key]
                #print(x,y)
    return np.dot(x,y)/(norm(x)*norm(y))   

In [248]:
#calculating global score with cosine coeficient
def cosine_movie_score(person, movie, Critiques):
    global_score = 0
    score = 0
    for i in Critiques.items():
        if not pd.isnull(i[1][movie]):
            global_score += i[1][movie] / (1+cosineCoefficient(Critiques[person], i[1]))
            score += 1 / (1+cosineCoefficient(Critiques[person], i[1]))
    return global_score / score

In [249]:
def CosineRecommend(person,Critiques):
    recommend_movies=[]
    for movie in get_unwatched_movies(person, Critiques):
        global_score = cosine_movie_score(person, movie, Critiques)
        recommend_movies.append((global_score,movie))
    print(recommend_movies)
    return sorted(recommend_movies)[-1][1]

In [250]:
CosineRecommend("Anne",Critiques)

[(3.7142857142857144, 'Snakes'), (4.0, 'Superman'), (3.4166666666666665, 'Night')]


'Superman'

### 3. Which songs will you recommend to Veronica? To Hailey?
Test again, all your previous functionalities through the following example with 8 music critics (Angelica, Bill, . . .) and 8 songs (Blues Travelers, Broken Bells, . . .). Which songs you can recommend to Veronica ? To Hailey?

In [251]:
#read the file
df = pd.read_csv(r'data\music.csv', delimiter=',',index_col=0)
df

,Angelica,Bill,Chan,Dan,Hailey,Jordyn,Sam,Veronica
Blues Traveler,3.5,2.0,5.0,3.0,NaN,NaN,5.0,3.0
Broken Bells,2.0,3.5,1.0,4.0,4.0,4.5,2.0,NaN
Deadmau5,NaN,4.0,1.0,4.5,1.0,4.0,NaN,NaN
Norah Jones,4.5,NaN,3.0,NaN,4.0,5.0,3.0,5.0
Phoenix,5.0,2.0,5.0,3.0,NaN,5.0,5.0,4.0
Slightly Stoopid,1.5,3.5,1.0,4.5,NaN,4.5,4.0,2.5
The Strokes,2.5,NaN,NaN,4.0,4.0,4.0,5.0,3.0
Vampire Weekend,2.0,3.0,NaN,2.0,1.0,4.0,NaN,NaN


In [252]:
#Convert data frame to dictionaries
music_critiques =df.to_dict()

In [253]:
music_critiques['Veronica']

{'Blues Traveler': 3.0,
 'Broken Bells': nan,
 'Deadmau5': nan,
 'Norah Jones': 5.0,
 'Phoenix': 4.0,
 'Slightly Stoopid': 2.5,
 'The Strokes': 3.0,
 'Vampire Weekend': nan}

#### (a) Songs Recommendation to Veronica 

##### Recommendation of songs based on NearestNeighbor

In [254]:
recommendNearestNeighbor('Veronica',music_critiques) 

The nearest neighbour of Veronica is Hailey 
The list of movies recommended is [('Broken Bells', 4.0), ('Deadmau5', 1.0), ('Vampire Weekend', 1.0)]


##### Recommendation of songs based on BestRecommend with Manhattan Distance 

In [255]:
Bestrecommend('Veronica',music_critiques)

[(3.243288590604027, 'Broken Bells'), (2.78125, 'Deadmau5'), (2.230769230769231, 'Vampire Weekend')]


'Broken Bells'

#####  Recommendation of songs based on BestRecommend with Euclidean  Distance 

In [256]:
Bestrecommend2('Veronica',music_critiques)

[(3.11679444654491, 'Broken Bells'), (2.8195523097548674, 'Deadmau5'), (2.2743715116949708, 'Vampire Weekend')]


'Broken Bells'

##### Recommendation of songs based on BestrecommendwithExp   

In [257]:
BestrecommendwithExp('Veronica',music_critiques)

[(3.7060486400373835, 'Broken Bells'), (1.907255466380703, 'Deadmau5'), (1.6353931490705687, 'Vampire Weekend')]


'Broken Bells'

##### Recommendation of songs based on  PearsonRecommend

In [258]:
PearsonRecommend('Veronica',music_critiques)

[(3.266135240747718, 'Broken Bells'), (3.679955010191387, 'Deadmau5'), (2.414911389671949, 'Vampire Weekend')]


'Deadmau5'

##### Recommendation of songs based on CosineRecommend 

In [259]:
CosineRecommend('Veronica',music_critiques)

[(3.0, 'Broken Bells'), (2.9, 'Deadmau5'), (2.4, 'Vampire Weekend')]


'Broken Bells'

#### Songs Recommendation to Hailey 

##### Recommendation of songs based on NearestNeighbor

In [260]:
recommendNearestNeighbor('Hailey',music_critiques) 

The nearest neighbour of Hailey is Veronica 
The list of movies recommended is [('Blues Traveler', 3.0), ('Phoenix', 4.0), ('Slightly Stoopid', 2.5)]


##### Recommendation of songs based on BestRecommend with Manhattan Distance 

In [261]:
Bestrecommend('Hailey',music_critiques)

[(3.59036030937559, 'Blues Traveler'), (4.1439456113154405, 'Phoenix'), (2.9330932044540363, 'Slightly Stoopid')]


'Phoenix'

#####  Recommendation of songs based on BestRecommend with Euclidean  Distance 

In [262]:
Bestrecommend2('Hailey',music_critiques)

[(3.5956193126180254, 'Blues Traveler'), (4.184141896852117, 'Phoenix'), (2.951062994818188, 'Slightly Stoopid')]


'Phoenix'

##### Recommendation of songs based on BestrecommendwithExp 

In [263]:
BestrecommendwithExp('Hailey',music_critiques)

[(3.374134370081063, 'Blues Traveler'), (4.1267130055552235, 'Phoenix'), (2.6063133402163063, 'Slightly Stoopid')]


'Phoenix'

##### Recommendation of songs based on  PearsonRecommend 

In [264]:
PearsonRecommend('Hailey',music_critiques)

[(3.527590349695004, 'Blues Traveler'), (4.047620727589134, 'Phoenix'), (3.1084718098167707, 'Slightly Stoopid')]


'Phoenix'

 ##### Recommendation of songs based on CosineRecommend  

In [265]:
CosineRecommend('Hailey',music_critiques)

[(3.5833333333333335, 'Blues Traveler'), (4.142857142857143, 'Phoenix'), (3.0714285714285716, 'Slightly Stoopid')]


'Phoenix'

### 4. Movies recommendations with critics(n) >=10 and movies(m)>=10
Elaborate your own example with n critics (n >= 10) and m movies (m >= 10) where the recommendations given to
the one of these critics (who has not seen at least half of the movies), based on at least 5 different similarity measures
(including Pearson and Cosine similarity measures), are completely different from each other. The number of missing
data (the empty cells in your evaluation matrix) should be between 30% and 50 % of the total of the matrix’s cells.

In this questions we have generate a data with 16 people and their critics on the 16 movies. The dataset consist of 83 empty cells which is 33% of the whole dataset.

In [266]:
#read the file
df = pd.read_csv(r'data/movie_gen.csv', delimiter=';',index_col=0)
df

,Fatmah,Hajra,Mahsan,Hassan,Asha,Said,Ahmed,Zuwena,Awena,Sabah,Seif,Salim,Anwar,Hannan,Ilham,Ariff
The lovers,3.5,NaN,3.0,3.5,4.0,3.0,2.5,3.0,4.0,2.5,3.0,4.5,3.5,3.0,NaN,4.0
Eat me more,4.0,NaN,3.0,4.0,3.5,2.5,3.6,4.5,3.0,NaN,2.5,3.5,3.6,3.0,4.5,NaN
Herois,3.5,NaN,NaN,2.5,NaN,NaN,2.0,NaN,NaN,1.5,2.0,3.5,3.0,4.0,3.0,3.5
Beginners,1.5,2.5,3.5,3.5,NaN,3.0,3.0,3.0,4.0,NaN,NaN,NaN,3.5,3.0,2.0,NaN
Vampires,4.0,NaN,4.0,3.0,3.0,NaN,3.5,NaN,NaN,3.0,3.0,NaN,2.0,2.0,1.5,3.0
Once Upon A time,4.5,3.0,2.0,4.5,NaN,NaN,4.0,3.5,2.0,NaN,3.5,2.0,4.0,1.5,3.5,NaN
My breath,NaN,NaN,3.5,4.0,4.0,NaN,3.5,NaN,NaN,2.0,2.0,4.0,1.5,3.0,3.0,2.5
Heartbeat,3.5,2.5,NaN,3.0,NaN,NaN,NaN,NaN,2.5,NaN,2.5,3.0,3.6,2.5,4.0,NaN
Stay Safe,NaN,NaN,2.0,NaN,NaN,2.0,3.0,4.5,NaN,NaN,3.0,3.0,4.0,NaN,4.5,3.0
The Origins,2.5,NaN,NaN,2.5,2.5,4.0,2.0,NaN,NaN,3.0,NaN,4.0,3.0,3.5,3.0,2.0


In [267]:
df.isna().sum().sum()

83

In [268]:
#Convert data frame to dictionaries
movies_critiques =df.to_dict()

In [269]:
movies_critiques['Hajra']

{'The lovers': nan,
 'Eat me more': nan,
 'Herois': nan,
 'Beginners': 2.5,
 'Vampires': nan,
 'Once Upon A time': 3.0,
 'My breath': nan,
 'Heartbeat': 2.5,
 'Stay Safe': nan,
 'The Origins': nan,
 'Story of my life': 3.0,
 'Teenagers': nan,
 'Neighbours': nan,
 'The Journey': 4.0,
 'The Hereafter': nan,
 'Before life': 1.5}

##### Movie Recommendations to Hajra with different measures
In this question we will recommends movies to Hajra, by using different measures


##### By using BestRecommend with Manhattan Distance

In [270]:
Bestrecommend('Hajra',movies_critiques)

[(3.3501620791015077, 'The lovers'), (3.2842368688423402, 'Eat me more'), (2.8392222472650355, 'Herois'), (2.8842633117220307, 'Vampires'), (2.9739884393063583, 'My breath'), (2.9815177179197114, 'Stay Safe'), (3.0411696015074505, 'The Origins'), (2.54811063498247, 'Teenagers'), (2.5724827908062067, 'Neighbours'), (2.5940318647192897, 'The Hereafter')]


'The lovers'

##### By using BestRecommend with Euclidean Distance

In [271]:
Bestrecommend2('Hajra',movies_critiques)

[(3.343319944930279, 'The lovers'), (3.3652713076239404, 'Eat me more'), (2.8602813182716402, 'Herois'), (2.8732798601686875, 'Vampires'), (2.970430175561473, 'My breath'), (3.0835038918287077, 'Stay Safe'), (2.99726482966636, 'The Origins'), (2.553131270998158, 'Teenagers'), (2.5361466265111505, 'Neighbours'), (2.5940408395793075, 'The Hereafter')]


'Eat me more'

##### By using BestrecommendwithExp

In [272]:
BestrecommendwithExp('Hajra',movies_critiques)

[(3.2943618645129833, 'The lovers'), (2.869179975455558, 'Eat me more'), (2.8335501332701836, 'Herois'), (2.903253688829444, 'Vampires'), (2.925190722560882, 'My breath'), (2.4655246997997415, 'Stay Safe'), (3.393852941033694, 'The Origins'), (2.366564743795617, 'Teenagers'), (2.7617962173056463, 'Neighbours'), (2.5847583656173465, 'The Hereafter')]


'The Origins'

##### By using PearsonRecommend  

In [273]:
PearsonRecommend('Hajra',movies_critiques)

[(3.387004429493516, 'The lovers'), (3.4813818648151527, 'Eat me more'), (3.06625101778652, 'Herois'), (2.9175372332033556, 'Vampires'), (3.0860416399106225, 'My breath'), (3.3350297791192967, 'Stay Safe'), (2.9469530431790147, 'The Origins'), (2.6654334842230445, 'Teenagers'), (2.395121359439874, 'Neighbours'), (2.591371095657789, 'The Hereafter')]


'Eat me more'

##### By using CosineRecommend

In [274]:
CosineRecommend('Hajra',movies_critiques)

[(3.357142857142857, 'The lovers'), (3.476923076923077, 'Eat me more'), (2.85, 'Herois'), (2.909090909090909, 'Vampires'), (3.0, 'My breath'), (3.2222222222222223, 'Stay Safe'), (2.909090909090909, 'The Origins'), (2.5625, 'Teenagers'), (2.4545454545454546, 'Neighbours'), (2.6153846153846154, 'The Hereafter')]


'Eat me more'

##### RESULTS OF MOVIES RECOMMENDATION TO HAJRA
The results of movie recommandations generated above, show that the Euclidean Distance recomendations, PearsonRecommend and CosineRecommends generate the best recommendation with movies **"Eat me more"**, while the rest generate different results.

The results generated is subjected to the nature of the dataset we have;
* Different users use different scales to the movies
* The data is spare

Therefore the Pearson and Cosine Similarity are the best measures to consider in our recommendations.
